In [70]:
import astropy.io.fits
import matplotlib.pyplot as plt
import numpy as np
import os
plt.rcParams["figure.figsize"] = (6,10)

In [ ]:


def plot_floyds_qc(fitsobject):

   
    #print (list(fitsobject[0].header.keys()))
    object  = fitsobject[0].header['OBJECT']
    dateobs = fitsobject[0].header['DATE-OBS']
    exptime = fitsobject[0].header['EXPTIME']   
    filename = fitsobject[0].header['ORIGNAME']
    wavelength = fitsobject[1].data['wavelength']

    flux = fitsobject[1].data['flux']
    background = fitsobject[1].data['background']

   

    plt.subplot(3,2,(1,2))

    c_flux='orange'
    c_background='blue'
    
    plt.plot (wavelength, background, label='Background Flux', c=c_background)
    plt.plot (wavelength, flux, label='Object Flux', c=c_flux)
    plt.title (f'{filename} \n {object}  • {dateobs} • {exptime}s')
    plt.xlabel ('Wavelength (Angstroms)')
    plt.ylabel ('Flux')
    plt.ylim (0, 1.2*np.max(flux))
    plt.legend()
    


    oxygen = 5577
    plt.subplot(3,2,3)
    plt.plot (wavelength, flux, label='Object Flux', c=c_flux)
    plt.plot (wavelength, background, label='Background Flux', c=c_background)
    plt.xlim (oxygen-200, oxygen+200)
    plt.axvline(oxygen, color='orange', linestyle='--', label='Sodium D2')
    plt.title ('Oxygen Line')
    plt.xlabel ('Wavelength (Angstroms)')
    plt.ylabel ('Flux')


    redoxygen = (6300+6360)/2
    plt.subplot(3,2,4)
    plt.plot (wavelength, flux, label='Object Flux', c=c_flux)
    plt.plot (wavelength, background, label='Background Flux', c=c_background)
    plt.xlim (redoxygen-200, redoxygen+200)
    plt.axvline(redoxygen, color='orange', linestyle='--', label='Sodium D2')
    plt.title ('Red Oxygen Line')
    plt.xlabel ('Wavelength (Angstroms)')
    plt.ylabel ('Flux')
   
    
    plt.tight_layout()

    plt.savefig (f'{filename}.png', dpi=300)
    plt.close()


def plot_2d_floyds_background(fitsobject):
    """
    Plot the 2D background data from a FITS file.
    
    Parameters
    ----------
    fitsobject : astropy.io.fits.HDUList
        The FITS file object containing the background data.

    """
    # print (fitsobject.info())
    background = fitsobject['BACKGROUND'].data
    image = fitsobject['SCI'].data
    
    object  = fitsobject['SCI'].header['OBJECT']
    dateobs = fitsobject['SCI'].header['DATE-OBS']
    exptime = fitsobject['SCI'].header['EXPTIME']   
    filename = fitsobject['SCI'].header['ORIGNAME']
   

    
    
    plt.figure()
    plt.subplot(4,1,1)
    mean, med, std = np.mean(image), np.median(background), np.std(background)
    plt.imshow(image, cmap='cubehelix', origin='lower', 
               vmin=med-3*std, vmax=med+10*std)
    plt.colorbar()
    plt.title('2D Image (fringe corrected)')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')

    plt.subplot(4,1,2)
    mean, med, std = np.mean(background), np.median(background), np.std(background)
    plt.imshow(background, cmap='cubehelix', origin='lower', 
               vmin=med-3*std, vmax=med+10*std)
    plt.colorbar()
    plt.title('2D Background Flux')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')

    plt.subplot(4,1,3)
    if 'FRINGE' in fitsobject:
        fringe = fitsobject['FRINGE'].data
        fringeoffset = fitsobject['SCI'].header.get('L1FRNGOF', 'N/A')
        mean, med, std = np.mean(fringe), np.median(fringe), np.std(fringe)
        plt.imshow(fringe, cmap='cubehelix', origin='lower', 
               vmin=med-3*std, vmax=med+10*std)
        plt.colorbar()
        plt.title(f'Fringe Map [offset={fringeoffset:5.2f}]'.format(fringeoffset=fringeoffset))
        plt.xlabel('Pixel X')
        plt.ylabel('Pixel Y')
    else:
        print (f"{filename}, target {object} has no fringe map")


    plt.subplot(4,1,4)
    mean, med, std = np.mean(image-background), np.median(background), np.std(background)
    plt.imshow(image-background, cmap='cubehelix', origin='lower', 
               vmin=med-3*std, vmax=med+10*std)
    plt.colorbar()
    plt.title('Background Subtracted Image')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')


    plt.tight_layout()
    plt.savefig(f'{filename}-background.png', dpi=300)
    plt.close()

def plot_2d_floyds_background_from_file(filename):
    """
    Plot the 2D background data from a FITS file.
    
    Parameters
    ----------
    filename : str
        The path to the FITS file containing the background data.
    """
    with astropy.io.fits.open(filename) as hdul:
       
        plot_2d_floyds_background(hdul)
        hdul.close()
        
def plot_floyds_qc_from_file(filename):
    """
    Plot the QC data from a FITS file.
    
    Parameters
    ----------
    filename : str
        The path to the FITS file containing the QC data.
    """
    with astropy.io.fits.open(filename) as hdul:
        
        plot_floyds_qc(hdul)
        hdul.close()
    


In [80]:
plot_floyds_qc_from_file('~/lco/banzai-floyds/coj_20250822/coj2m002-en12-20250716-0014-e91-1d.fits.fz')
plot_2d_floyds_background_from_file('~/lco/banzai-floyds/coj_20250822/coj2m002-en12-20250716-0014-e91-2d.fits.fz')

In [77]:
def list_1d_fits_fz_files(directory):
    return [f'{directory}/{f}' for f in os.listdir(directory) if f.endswith('-1d.fits.fz')]

def list_2d_fits_fz_files(directory):
    return [f'{directory}/{f}' for f in os.listdir(directory) if f.endswith('-2d.fits.fz')]

allfiles = list_1d_fits_fz_files('/Users/dharbeck/lco/banzai-floyds/coj_20250822/')
print (allfiles)
for file in allfiles:
    plot_floyds_qc_from_file(file)





['/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250719-0002-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250822-0001-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250716-0014-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250718-0011-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250719-0011-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250718-0001-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250719-0014-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250719-0001-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250718-0012-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-en12-20250716-0017-e91-1d.fits.fz', '/Users/dharbeck/lco/banzai-floyds/coj_20250822//coj2m002-e

In [81]:
all2dfiles = list_2d_fits_fz_files('/Users/dharbeck/lco/banzai-floyds/coj_20250822/')
for file in all2dfiles:
    plot_2d_floyds_background_from_file(file)
       

coj2m002-en12-20250813-0002-e00.fits, target NGC6814 has no fringe map
coj2m002-en12-20250817-0004-e00.fits, target ESO_511-30 has no fringe map
coj2m002-en12-20250812-0002-e00.fits, target ESO_511-30 has no fringe map
coj2m002-en12-20250812-0010-e00.fits, target Fairall9 has no fringe map
coj2m002-en12-20250812-0011-e00.fits, target Fairall9 has no fringe map
coj2m002-en12-20250812-0003-e00.fits, target ESO_511-30 has no fringe map
coj2m002-en12-20250815-0002-e00.fits, target PDS 456 has no fringe map
coj2m002-en12-20250817-0005-e00.fits, target ESO_511-30 has no fringe map
coj2m002-en12-20250811-0011-e00.fits, target SN2025tam has no fringe map
coj2m002-en12-20250812-0014-e00.fits, target SN2025tis has no fringe map
coj2m002-en12-20250813-0001-e00.fits, target NGC6814 has no fringe map
coj2m002-en12-20250811-0001-e00.fits, target at2020fvu has no fringe map
coj2m002-en12-20250811-0008-e00.fits, target SN2025qpk has no fringe map
coj2m002-en12-20250818-0011-e00.fits, target FEIGE110 h